In [ ]:
import os
import re
from collections import defaultdict

In [ ]:
import pandas as pd

data = defaultdict(lambda: {'topic': '', 'content': '', 'messages': [], 'month': ''})

for file_path in os.listdir("cleaned_intercom_messages"):
    csv = pd.read_csv(f"cleaned_intercom_messages/{file_path}")
    csv = csv[csv['clean_body'].notna()]
    bodies = csv["clean_body"]
    ids = zip(csv["key_intercomconversation"], csv["key_iteration"])
    for id, body in zip(ids, bodies):
        m = re.search(" Selected category: (.+) Content: (.+) uid:", body)
        if m is not None:
            data[id]['topic'] = m.group(1)
            data[id]['content'] = m.group(2)
        else:
            data[id]['messages'].append(body)
            if data[id]['month'] == '':
                data[id]['month'] = file_path.split("-")[1]

df = pd.DataFrame.from_dict(data, orient="index")
print(len(df[df["topic"] == '']))
print(len(df[df["topic"] != '']))

In [ ]:
df.head()

In [ ]:
new_ids = ["_".join([str(tup[0]), str(tup[1])]) for tup in df.index]

In [ ]:
df.index = new_ids

In [ ]:
df.head()

In [ ]:
def get_monthly_data(original_data: pd.DataFrame, preprocessed_data: dict, month: str) -> dict:
    ids = original_data[original_data['month'] == month].index
    monthly_data = {}
    for _id in ids:
        if _id in preprocessed_data.keys():
            monthly_data[_id] = preprocessed_data[_id]
        
    return monthly_data

In [ ]:
import json

In [ ]:
with open("english_texts_by_id.json") as f:
    preprocessed = json.loads(f.read())

In [ ]:
len(preprocessed)

In [ ]:
months = []

for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']:
    months.append(get_monthly_data(df, preprocessed, month))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
import time
from datetime import timedelta

In [ ]:
models = []

print("Start:", str(timedelta(seconds=time.time()+7200)).split(", ")[1])

for i, month in enumerate(months):
    lists = month.values()
    X = TfidfVectorizer().fit_transform(lists)
    lda = LatentDirichletAllocation(n_components=7)
    lda.fit(X)
    models.append(lda)
    print(f"Month {i} done at:", str(timedelta(seconds=time.time()+7200)).split(", ")[1])

In [ ]:
lists = months[2].values()
X = TfidfVectorizer().fit_transform(lists)
probs = models[2].transform(X)

In [ ]:
import numpy as np

In [ ]:
list(lists)[0]

In [ ]:
topics_2 = {}

for i in range(7):
    topics_2[i] = []

vals = list(lists)
    
for i, prob in enumerate(probs):
    index = np.argmax(prob)
    topics_2[index].append(vals[i])

In [ ]:
for k, v in topics_2.items():
    print(k, len(v))

In [ ]:
topics_2[5]

In [ ]:
model = LatentDirichletAllocation(n_components=25)
X_new = TfidfVectorizer().fit_transform(topics_0[6])
model.fit(X_new)

In [ ]:
probabilities = model.transform(X_new)

In [ ]:
topics_0_new = {}

for i in range(25):
    topics_0_new[i] = []

vals = list(topics_0[6])
    
for i, prob in enumerate(probabilities):
    index = np.argmax(prob)
    topics_0_new[index].append(vals[i])

In [ ]:
for k, v in topics_0_new.items():
    print(k, len(v))

In [ ]:
topics_0_new[0]

In [ ]:
topics_0[6][0]

In [ ]:
from collections import defaultdict

counter = defaultdict(lambda: defaultdict(int))
n = X.shape[0]
transformed = models[2].transform(X[:n])
for i in range(n):
    counted_words = defaultdict(int)
    text = list(months[2].values())[i]
    topic = transformed[i].argmax()
    for word in text.split():
        counter[topic][word] += 1

dict2 = dict()

for topic in counter:
    for word in counter[topic]:
        all_contain = True
        for t in counter:
            all_contain = all_contain and word in counter[t]
        
                
    items = list(counter[topic].items())
    items.sort(key=lambda x: x[1], reverse=True)
    dict2[topic] = items[0:1000]
    
dict2

In [ ]:
for k, v in dict2.items():
    for word in v:
        if 'corona' in word[0] or 'covid' in word[0]:
            print(k, word)

---

### OTSIME KOROONAT

In [ ]:
corona = {}

for i in range(10):
    corona[i] = []

In [ ]:
for i in range(10):
    lists = months[i].values()
    X = TfidfVectorizer().fit_transform(lists)
    probs = models[i].transform(X)
    
    topics = {}

    for idx in range(7):
        topics[idx] = []

    vals = list(lists)

    for idx, prob in enumerate(probs):
        index = np.argmax(prob)
        topics[index].append(vals[idx])
        
        
    counter = defaultdict(lambda: defaultdict(int))
    n = X.shape[0]
    transformed = models[i].transform(X[:n])
    for idx in range(n):
        counted_words = defaultdict(int)
        text = list(months[i].values())[idx]
        topic = transformed[idx].argmax()
        for word in text.split():
            counter[topic][word] += 1

    dict2 = dict()

    for topic in counter:
        for word in counter[topic]:
            all_contain = True
            for t in counter:
                all_contain = all_contain and word in counter[t]


        items = list(counter[topic].items())
        items.sort(key=lambda x: x[1], reverse=True)
        dict2[topic] = items[0:2000]
        
    for k, v in dict2.items():
        for word in v:
            if 'corona' in word[0] or 'covid' in word[0]:
                corona[i].append([k, word])

In [ ]:
corona

In [ ]:
corona_counts = {}

for i in range(10):
    corona_counts[i] = {}
    for topic in corona[i]:
        topic_nr = topic[0]
        c = topic[1]
        if topic_nr in corona_counts[i].keys():
            corona_counts[i][topic_nr] += c[1]
        else:
            corona_counts[i][topic_nr] = c[1]

In [ ]:
corona_counts

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
df = pd.DataFrame.from_dict(corona_counts).fillna(0)

In [ ]:
df = df.astype(int)
df

In [ ]:
import seaborn as sns

In [ ]:
["y"]*3+["z"]*3

In [ ]:
df.loc[5]

In [ ]:
list(df[0])+list(df[1])

In [ ]:
data = pd.DataFrame(zip(["january", "february", "march", "april", "may", "june", "july", "august", "september", "october"]*6, ["0"]*6+["1"]*6+["2"]*6+["3"]*6+["5"]*6+["6"]*6, list(df.loc[0])+list(df.loc[1])+list(df.loc[2])+list(df.loc[3])+list(df.loc[5])+list(df.loc[6])), columns=["months", "topic", "results"])
plt.figure(figsize=(10, 6))
sns.barplot(x="months", y="results", hue="topic", data=data)

In [ ]:
data = pd.DataFrame(zip(["january", "february", "march", "april", "may", "june", "july", "august", "september", "october"]*6, ["0"]*6+["1"]*6+["2"]*6+["3"]*6+["5"]*6+["6"]*6, list(df[0])+list(df[1])+list(df[2])+list(df[3])+list(df[5])+list(df[6])+list(df[7])+list(df[8])+list(df[9])), columns=["months", "topic", "results"])
plt.figure(figsize=(10, 6))
sns.barplot(x="months", y="results", hue="topic", data=data)

In [ ]:
df.loc[3]

In [ ]:
df2 = df.drop(3, axis=1)
df2

In [ ]:
df.T.plot(kind='bar')

In [ ]:
plt.bar(df.index, df[2])

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(12, 5))
plt.subplots_adjust(top=0.99, bottom=0.01, hspace=0.35, wspace=0.7)

current = 2
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October"]
        
for row in ax:
    for col in row:
        col.bar(df.index, df[current])
        col.set_xticks(range(7))
        col.set_xlabel("Topics")
        col.set_ylabel("Frequency")
        col.title.set_text(months[current])
        current += 1

In [ ]:
data = pd.DataFrame(zip(["january", "february", "march", "may", "june", "july", "august", "september", "october"]*9, ["0"]*9+["1"]*9+["2"]*9+["3"]*9+["5"]*9+["6"]*9, list(df2.loc[0])+list(df2.loc[1])+list(df2.loc[2])+list(df2.loc[3])+list(df2.loc[5])+list(df2.loc[6])), columns=["months", "topic", "results"])
plt.figure(figsize=(17, 7))
sns.barplot(x="months", y="results", hue="topic", data=data)

In [ ]:
data

In [ ]:
from collections import defaultdict

counter = defaultdict(lambda: defaultdict(int))
n = X_new.shape[0]
transformed = model.transform(X_new[:n])
for i in range(n):
    counted_words = defaultdict(int)
    text = topics_0[6][i]
    topic = transformed[i].argmax()
    for word in text.split():
        counter[topic][word] += 1

dict2 = dict()

for topic in counter:
    for word in counter[topic]:
        all_contain = True
        for t in counter:
            all_contain = all_contain and word in counter[t]
        
                
    items = list(counter[topic].items())
    items.sort(key=lambda x: x[1], reverse=True)
    dict2[topic] = items[0:10]
    
dict2

In [ ]:
counter.keys()